# Sudoku

A popular number game/puzzle whose objective is to fill each of the empty cells in a grid with the correct number using the given initial clues. The classic sudoku game involves a $9$ x $9$ grid in which the grid is divided into $9$ blocks, and each block contains $9$ cells. 

Note that I will generalize this to be in a form of $n$ x $n$ grid where $n$ is a perfect square (e.g., 4, 9, and 16).

## Solving Sudoku Puzzle

Every empty cell must be filled without violating the following rules.
1. Each row must contain the numbers without repetitions
2. Each column must contain the numbers without repetitions
3. Each block must contain the numbers without repetitiions

where in this case the numbers go from $1$ to $n$.

One of the common ways to solve sudoku is to use a backtracking algorithm which tries all the possible combination of the board. This, howevers, can be inefficient. 

Therefore, in this project, I will be using Linear Optimization algorithm to solve sudoku puzzles.

In [1]:
import numpy as np

In [2]:
# nxn sudoku class
class Sudoku:

    def __init__(self, n = 9):
        # initialize
        self.n = n
        self.sqrt_n = (int) (np.sqrt(n))
        self.puzzle = self.get_new_puzzle()

    def get_new_puzzle(self):
        def get_completed_board():
            def get_non_repeat_index(row, col):
                # return the index i of a 1d choice array so that the number at i 
                # does not violate the sudoku board rules when placed at row, col
                return (self.sqrt_n*row + (row // self.sqrt_n) + col) % self.n
            # get a choices (i.e., 1, 2, ..., n) to be used in a random order
            choices = np.random.choice(range(1, self.n + 1), self.n, replace = False)
            # create and return the board
            return np.array([ 
                [ choices[get_non_repeat_index(row, col)] for col in range(self.n) ] 
                for row in range(self.n) 
            ])
        # get the completed board
        completed_board = get_completed_board()
        # use the completed board to generate a puzzle 
        # this guarantees at least one solution
        puzzle = np.copy(completed_board)
        # randomly remove cells until no more cells can be removed
        # without losing the uniqueness of the solution
        has_unique_solution = False
        while (not has_unique_solution):
            row = np.random.randint(self.n)
            col = np.random.randint(self.n)
            if (puzzle[row, col] != 0):
                # back up and remove the number from the cell
                number = puzzle[row, col]
                puzzle[row, col] = 0
                solutions = []
                # try to solve the board using backtracking algorithm
                self.solve_backtracking(puzzle, solutions, max_number_of_needed_solutions = 2)
                # stop if the solution is no longer unique
                if (len(solutions) > 1):
                    # put the number back so that the solution is unique
                    puzzle[row, col] = number
                    break
        return puzzle

    def solve_linear_optimization(self):
        # to be implemented
        pass

    def solve_backtracking(self, board, solutions, max_number_of_needed_solutions = -1):
        # stop in case the max number of needed solutions is satisfied
        if (len(solutions) == max_number_of_needed_solutions): return
        # traverse the board
        for row in range(self.n):
            for col in range(self.n):
                if (board[row, col] == 0):
                    # try to find the number to fill
                    for number in range(1, self.n + 1):
                        if (self.valid_cell(row, col, number, board)):
                            board[row, col] = number
                            self.solve_backtracking(board, solutions, max_number_of_needed_solutions)
                            board[row, col] = 0
                    return
        # collect the solutions
        solutions.append(np.copy(board).astype(int))

    def valid_cell(self, row, col, number, board):
        # check if number is in the given row
        in_row = number in board[row, :]
        # check if number is in the given col
        in_col = number in board[:, col]
        # get starting points of the block where the given row and col are in
        i = row - (row % self.sqrt_n)
        j = col - (col % self.sqrt_n)
        # check if number is in the given block
        in_block = number in board[i:(i + self.sqrt_n), j:(j + self.sqrt_n)]
        # valid if number is neither in row, col, nor block
        return not (in_row or in_col or in_block)

    def valid_board(self, board):
        # traverse the board
        for row in range(self.n):
            for col in range(self.n):
                if (board[row, col] != 0):
                    # back up and remove the number from the cell
                    number = board[row, col]
                    board[row, col] = 0
                    # check if the board is valid if the number is placed back to the cell
                    if (self.valid_cell(row, col, number, board)):
                        board[row, col] = number
                    else:
                        return False
        # board is valid if all cells are valid
        return True
    
    def accepted(self, solution):
        # solution is accepted if the solution is valid and contains no empty cells
        return self.valid_board(solution) and 0 not in solution

## Solving Sudoku Puzzle using Linear Optimization

Note that because sudoku only allows integer to be put into cells, a sudoku puzzle is therefore a mixed integer optimization problem.

However, unlike a normal linear optimization problem, the problem of sudoku puzzle does not have an objective function to be optimized. This is because as long as all the constraints are satisfied, a solution is considered to be valid. In other words, the objective function can be any arbitrary function which, in this case, I will be using zero as the objective function for simplicity. 

Now, let consider the decision variables.

Since a number must be unique in a row, a column, and a block, the decision variables cannot simply be each of the cells in the $n$ x $n$ board. 

Instead, let utilize the structure of $n$ x $n$ x $n$ grid where each of the cells is either $0$ or $1$ representing the existence of the corresponding value at the corresponding cell in the $n$ x $n$ sudoku board. This ways we can represent all the combination of the value, row, and column.

So, we have 

$$
x_{k, i, j} = 
\begin{cases}
    1 & \quad \text{if number $k$ is in row $i$, column $j$}, \\
    0 & \quad \text{otherwise}
\end{cases}
$$

Next, let define all the constraints.

1. Each row must contain the numbers without repetitions

$$
\sum_{j = 1}^{n} x_{k, i, j} = 1, \quad \quad \forall i, k \in [1, n]
$$

2. Each column must contain the numbers without repetitions

$$
\sum_{i = 1}^{n} x_{k, i, j} = 1, \quad \quad \forall j, k \in [1, n]
$$

3. Each block must contain the numbers without repetitiions

$$
\sum_{i = 1}^{n}\sum_{j = 1}^{n} x_{k, i + u, j + v} = 1, \quad \quad \forall k \in [1, n] \text{ and } u, v \in \{0, \sqrt{n}, 2\sqrt{n}, ..., n - \sqrt{n} \}
$$

This is, however, still not enough because we also want to ensure that only one number can be in a cell and each of the cells in our $n$ x $n$ x $n$ grid must be either 0 or 1. So, we have 

4. Each cell must contain only one number

$$
\sum_{k = 1}^{n} x_{k, i, j} = 1, \quad \quad \forall i, j \in [1, n]
$$

5. Each cell in the $n$ x $n$ x $n$ grid must be either 0 or 1

$$
0 \leqslant x_{k, i, j} \leqslant 1, \quad \quad \forall i, j, k \in [1, n]
$$ 

Finally, since we will use this to solve sudoku puzzles, we also need to ensure that the position of the initial clues (i.e., the known cells or the starting numbers that are given) are fixed. So, we have 

6. Each initial clue must be fixed

$$
x_{C_{i, j}, i, j} = 1, \quad \quad \forall C_{i, j}
$$

where $C_{i, j}$ is an initial clue that is not $0$ at row $i$ and column $j$

Now, let implement this using Python cvxpy

In [3]:
import cvxpy as cp

In [4]:
def solve_linear_optimization(self):
    x = { i : cp.Variable(shape = (self.n, self.n), integer = True) for i in range(self.n) }
    def get_constraints():
        # ensure a number can appear only once in a row/column
        row_constraints, col_constraints = map(list, zip(*[
            (cp.sum(x[k], axis = 1) == 1, cp.sum(x[k], axis = 0) == 1) 
            for k in range(self.n)
        ]))
        # ensure a number can appear only once in a sqrt(n) x sqrt(n) block
        block_constraints = []
        for k in range(self.n):
            for u in range(0, self.n - self.sqrt_n + 1, self.sqrt_n):
                for v in range(0, self.n - self.sqrt_n + 1, self.sqrt_n):
                    block_constraints.append(sum([
                        x[k][i + u, j + v] 
                        for i in range(self.sqrt_n) for j in range(self.sqrt_n)
                    ]) == 1)
        # ensure a number in the range and that every cell contains only one number
        cell_constraints = list(sum([
            (0 <= x[k], x[k] <= 1) 
            for k in range(self.n)
        ], ())) + [
            sum([x[k][i, j] for k in range(self.n)]) == 1 
            for i in range(self.n) for j in range(self.n)
        ]
        # ensure the positions of known cells (initial clues) from the puzzle are fixed
        known_cell_constraints = [
            x[self.puzzle[i, j] - 1][i, j] == 1 
            for i in range(self.n) for j in range(self.n) 
            if (self.puzzle[i,j] != 0)
        ]
        return row_constraints + col_constraints + block_constraints + known_cell_constraints + cell_constraints
    def get_answer(variables):
        answer = np.copy(self.puzzle)
        for k, variable in enumerate(variables, start = 1):
            # find indexes where the cells are 1
            indexes = np.array(np.where(variable.value == 1))
            # for each 1, fill the corresponding cell in the answer with k
            for idx in range(len(indexes[0])):
                row, col = indexes[:, idx]
                if (answer[row, col] == 0):
                    answer[row, col] = k
        return answer
    # define problem
    prob = cp.Problem(
        # use zero as the objective function
        objective = cp.Minimize(0),
        # get and set all the constraints
        constraints = get_constraints()
    )
    # solve
    prob.solve(solver = cp.GLPK_MI)
    # return the answer
    return get_answer(prob.variables())

Sudoku.solve_linear_optimization = solve_linear_optimization

In [5]:
# create sudoku puzzle
sudoku = Sudoku()
sudoku.puzzle

array([[0, 0, 0, 0, 0, 0, 4, 1, 0],
       [0, 0, 0, 4, 1, 9, 0, 3, 0],
       [0, 0, 9, 2, 0, 6, 7, 8, 0],
       [3, 0, 7, 8, 5, 0, 0, 9, 0],
       [0, 5, 0, 0, 9, 2, 3, 6, 0],
       [1, 9, 0, 0, 6, 0, 0, 0, 4],
       [0, 0, 8, 5, 4, 1, 0, 0, 3],
       [5, 0, 1, 0, 0, 0, 6, 7, 8],
       [0, 0, 3, 6, 7, 8, 0, 0, 0]])

In [6]:
# solve using linear optimization
solution = sudoku.solve_linear_optimization()
solution

array([[2, 3, 6, 7, 8, 5, 4, 1, 9],
       [7, 8, 5, 4, 1, 9, 2, 3, 6],
       [4, 1, 9, 2, 3, 6, 7, 8, 5],
       [3, 6, 7, 8, 5, 4, 1, 9, 2],
       [8, 5, 4, 1, 9, 2, 3, 6, 7],
       [1, 9, 2, 3, 6, 7, 8, 5, 4],
       [6, 7, 8, 5, 4, 1, 9, 2, 3],
       [5, 4, 1, 9, 2, 3, 6, 7, 8],
       [9, 2, 3, 6, 7, 8, 5, 4, 1]])

In [7]:
# solve using backtracking
solutions = []
sudoku.solve_backtracking(sudoku.puzzle, solutions)
solutions

[array([[2, 3, 6, 7, 8, 5, 4, 1, 9],
        [7, 8, 5, 4, 1, 9, 2, 3, 6],
        [4, 1, 9, 2, 3, 6, 7, 8, 5],
        [3, 6, 7, 8, 5, 4, 1, 9, 2],
        [8, 5, 4, 1, 9, 2, 3, 6, 7],
        [1, 9, 2, 3, 6, 7, 8, 5, 4],
        [6, 7, 8, 5, 4, 1, 9, 2, 3],
        [5, 4, 1, 9, 2, 3, 6, 7, 8],
        [9, 2, 3, 6, 7, 8, 5, 4, 1]])]

In [8]:
# since the solution is unique, this should return True
(solution == solutions[0]).all()

True

In [9]:
# check whether the solution is correct and should be accepted or not
sudoku.accepted(solution)

True